In [ ]:
import rocketpy
from rocketpy import Environment, Rocket, Flight, SolidMotor 
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import datetime 

# Environment

In [ ]:
env = Environment(
    latitude = -21.90795,
    longitude = -48.96156,
    elevation = 495,
)

date = datetime.date.today() + datetime.timedelta(days=1)

env.set_date(
    (date.year, date.month, date.day, 12), 
    timezone="America/Sao_Paulo"
)


env.set_atmospheric_model(
    type="custom_atmosphere",
    wind_u=[
        (0, 2.778), 
        (2000, 2.778) 
    ],
    wind_v=[
        (0, 0), 
        (2000, 0), 
    ],
)
env.max_expected_height = 2000


In [ ]:
env.info()

# Solid Motor

In [ ]:
Rocket_motor_1km = SolidMotor(
    thrust_source="thrust_time.csv",
    dry_mass = 2.30,
    dry_inertia = (0.0172, 0.0172, 0.0027),
    nozzle_radius= 14.7/1000,
    grain_number= 2,
    grain_density = 1700,
    grain_outer_radius = 36 / 1000,
    grain_initial_inner_radius = 7.5 / 1000,
    grain_initial_height = 110 / 1000,
    grain_separation = 1 / 1000,
    grains_center_of_mass_position = 150.5 / 1000,
    center_of_dry_mass_position = 120.7 / 1000,
    nozzle_position= 0,
    burn_time= 3.5,
    throat_radius= 5.5/1000,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
Rocket_motor_1km.info()

# Rocket

In [ ]:
df = pd.read_csv("cd_mach.DAT")

cd_off_func = interp1d(df["mach"], df["cd_off"], fill_value="extrapolate")
cd_on_func  = interp1d(df["mach"], df["cd_on"],  fill_value="extrapolate")

In [ ]:
Rocket_1km = Rocket(
    radius= 37.50/1000,
    mass= 5.2,
    inertia=(0.534, 0.534, 0.031), 
    power_off_drag= cd_off_func,
    power_on_drag= cd_on_func,
    center_of_mass_without_motor= 1.0, 
    coordinate_system_orientation="tail_to_nose",
)

Rocket_1km.add_motor(Rocket_motor_1km, position = -26 / 1000)

## AED

In [ ]:
nose_cone = Rocket_1km.add_nose(
    length= 210.647/1000, kind="elliptical", position= 1536.189 /1000,
)

fin_set = Rocket_1km.add_trapezoidal_fins(
    n=4,
    root_chord = 125/ 1000,
    tip_chord = 65 / 1000,
    span= 65 / 1000,
    position= 125 / 1000,
    cant_angle = 0,
)

In [ ]:
fin_set.all_info()
Rocket_1km.plots.static_margin()

## REC

In [ ]:
def usar_paraquedas():
    Rocket_1km.add_parachute(
        name="main",
        cd_s=1.5*np.pi,
        trigger="apogee",
        sampling_rate=100
        )

# Esquema Do Foguete

In [ ]:
Rocket_1km.draw()

# Flight

In [ ]:
test_flight_balistic = Flight(
    rocket=Rocket_1km, environment=env, rail_length=4, inclination=80, heading=0
    )

usar_paraquedas()

test_flight = Flight(
    rocket=Rocket_1km, environment=env, rail_length=4, inclination=80, heading=0
    )

test_flight.plots.trajectory_3d()
test_flight.prints.apogee_conditions()
test_flight_balistic.plots.trajectory_3d()

### Export data

In [ ]:
test_flight.export_data(
    "Rocket_1km.csv",
    "angle_of_attack",
    "mach_number",
    time_step=1.0,
)

test_flight_balistic.export_data(
    "Rocket_1km.csv",
    "angle_of_attack",
    "mach_number",
    time_step=1.0,
)

test_flight.plots.trajectory_3d(filename="trajectory_plot_1km.jpg")
test_flight_balistic.plots.trajectory_3d(filename="trajectory_plot_1km_balistic.jpg")